<a href="https://colab.research.google.com/github/danling-northwestern/stat303-2project/blob/main/Assignment_B_041923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions {-}

1. You may talk to a friend, discuss the questions and potential directions for solving them. However, you need to write your own solutions and code separately, and not as a group activity. 

2. Do not write your name on the assignment.

3. Write your code in the *Code* cells and your answer in the *Markdown* cells of the Jupyter notebook. Ensure that the solution is written neatly enough to understand and grade.

4. Use [Quarto](https://quarto.org/docs/output-formats/html-basics.html) to print the *.ipynb* file as HTML. You will need to open the command prompt, navigate to the directory containing the file, and use the command: `quarto render filename.ipynb --to html`. Submit the HTML file.

5. The assignment is worth 100 points, and is due on **Sunday, 23rd April 2023 at 11:59 pm**. 

6. **Five points are properly formatting the assignment**. The breakdown is as follows:
- Must be an HTML file rendered using Quarto (2 pts). *If you have a Quarto issue, you must mention the issue & quote the error you get when rendering using Quarto in the comments section of Canvas, and submit the ipynb file. If your issue doesn't seem genuine, you will lose points.* 
- There aren’t excessively long outputs of extraneous information (e.g. no printouts of entire data frames without good reason, there aren’t long printouts of which iteration a loop is on, there aren’t long sections of commented-out code, etc.) (1 pt)
- Final answers of each question are written in Markdown cells (1 pt).
- There is no piece of unnecessary / redundant code, and no unnecessary / redundant text (1 pt)

7. For all questions on cross-validation, you must use `sklearn` functions.

## Degrees of freedom
Find the number of degrees of freedom of the followng models. Exclude the intercept when counting the degrees of freedom. You may either show your calculation, or explain briefly how you are computing the degrees of freedom.

### Quadratic spline
A model with one predictor, where the predictor is transformed into a quadratic spline with 5 knots

*(2 points)*

2+5=7

### Natural cubic splines
A model with one predictor, where the predictor is transformed into a natural cubic spline with 4 knots

*(2 points)*

3+4-2=5

### Generalized additive model
A model with four predictors, where the transformations of the respective predictors are (i) cubic spline transformation with 3 knots, (ii) log transformation, (iii) linear spline transformation with 2 knots, (iv) polynomial transformation of degree 4.

*(4 points)*

(i) 3+3=6
(ii) 1 (log, not doing anything)
(iii) 1+2=3
(iv) 4 because there is no knot
<br>total: 6+1+3+4=14


## Number of knots
Find the number of knots in the following spline transformations, if each of the transformation corresponds to 7 degrees of freedom (excluding the intercept).

### Cubic splines
Cubic spline transformation 

*(1 point)*

7-3=4

### Natural cubic splines
Natural cubic spline transformation 

*(1 point)*

7+2-3=6

### Degree 4 spline
Spline transformation of degree 4

*(1 point)*

7-4=3

## Regression problem

Read the file *investment_clean_data.csv*. This data is a cleaned version of the file *train.csv* in last quarter's regression [prediction problem](https://www.kaggle.com/competitions/data-science-2-linear-regression-2023-bank-loans). Refer to the link for description of variables. It required some effort to get a RMSE of less than 650 with linear regression. In this question, we'll use MARS / natural cubic splines to get a RMSE of less than 350 with relatively less effort.

### Data preparation

Prepare the data for modeling as follows:

1. Use the Pandas function `get_dummies()` to convert all the categorical predictors to dummy variables. 

2. Using the `sklearn` function `train_test_split`, split the data into 20% test and 80% train. Use `random_state = 45`.

*Note:*

*A. The function `get_dummies()` can be used over the entire DataFrame. Don't convert the categorical variables individually.*

*B. The MARS model does not accept categorical predictors, which is why the conversion is done.*

*C. The response is `money_made_inv`*

*(2 points)*

In [ ]:
# earth for google colab
!git clone --single-branch --branch v0.2dev https://github.com/scikit-learn-contrib/py-earth.git
%cd py-earth
!python setup.py install --cythonize

Cloning into 'py-earth'...
remote: Enumerating objects: 3303, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 3303 (delta 21), reused 17 (delta 17), pack-reused 3278
Receiving objects: 100% (3303/3303), 13.03 MiB | 31.85 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
/content/py-earth
Compiling pyearth/_basis.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_record.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_pruning.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_forward.pyx because it changed.
Compiling pyearth/_knot_search.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_qr.pyx because it depends on ./pyearth/_types.pxd.
[1/6] Cythonizing pyearth/_forward.pyx
/usr/local/lib/python3.9/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from patsy import dmatrix

In [ ]:
data = pd.read_csv('investment_clean_data.csv')
data.head()

,id,money_made_inv,acc_now_delinq,acc_open_past_24mths,addr_state,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,...,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status,earliest_cr_line_year,earliest_cr_line_month,last_credit_pull_d_year,last_credit_pull_d_month
0,3819,-2787.380000,0,2,CT,21120.0,Individual,3662,86.1,0,...,C1,36,0,65921,0.0,Verified,2010,9,2010,9
1,3820,2.436708,0,3,CA,32400.0,Individual,2973,30.2,0,...,C1,36,0,17840,0.0,Not Verified,1975,1,1975,1
2,3821,-3.773192,0,3,NY,30251.0,Individual,1983,46.2,0,...,B5,36,0,19829,0.0,Source Verified,2010,10,2010,10
3,3822,-7951.970000,0,1,NJ,66976.0,Individual,1638,77.4,0,...,C1,60,0,9830,0.0,Source Verified,2012,3,2012,3
4,3823,-8058.760000,0,2,MO,125000.0,Individual,577,0.0,0,...,A2,60,0,6924,0.0,Source Verified,1982,7,1982,7


In [ ]:
# term should be a cat variable
data.term = data.term.astype('str')

In [ ]:
# get dummies
data_dum = pd.get_dummies(data, drop_first=True)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_dum.drop(columns = ['money_made_inv']), data_dum.money_made_inv, test_size=0.2, random_state=45)

### Optimal MARS degree
Use $5$-fold cross validation to find the optimal degree of the MARS model to predict `money_made_inv` based on all the predictors in the dataset.

*(4 points)*

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

def rmse(true, pred):
    rmse = mean_squared_error(true, pred, squared=False)
    return rmse

In [ ]:
from pyearth import Earth

parameters={'max_degree':[1,2,3,4]}
mars = Earth()
gs = GridSearchCV(mars, parameters,cv=5,scoring = 'neg_root_mean_squared_error')
result=gs.fit(X_train,y_train)



In [ ]:
df=pd.DataFrame(gs.cv_results_['params'])
df['mean_test_score']=gs.cv_results_['mean_test_score']
df['std_test_score']=gs.cv_results_['std_test_score']
df['rank_test_score']=gs.cv_results_['rank_test_score']
df.sort_values('rank_test_score')

,max_degree,mean_test_score,std_test_score,rank_test_score
3,4,-633.838734,63.205249,1
2,3,-640.890394,75.870473,2
1,2,-770.351108,87.126270,3
0,1,-1483.795472,75.710161,4


### Fitting MARS model

With the optimal degree identified in the previous question, fit a MARS model. Print the model summary. What is the degree of freedom of the model (excluding the intercept)? 

*(1 + 1 + 2 points)*

# Check this!!!
The optimal degree identified in the previous question is 4. The degree of freedom of the model is 4+1=5. 

In [ ]:
# so I will set max_degree = 4 
mars1 = Earth(feature_importance_type='gcv', max_degree = 4)
mars1.fit(X_train,y_train)

Earth(feature_importance_type='gcv', max_degree=4)

In [ ]:
print(mars1.summary())

Earth Model
------------------------------------------------------------------------
Basis Function                                    Pruned  Coefficient   
------------------------------------------------------------------------
(Intercept)                                       No      -15.4464      
h(out_prncp_inv-640.27)                           No      -1.09564      
h(640.27-out_prncp_inv)                           Yes     None          
h(int_rate-30.75)*h(out_prncp_inv-640.27)         No      0.718987      
h(30.75-int_rate)*h(out_prncp_inv-640.27)         No      -0.0148913    
loan_amnt                                         No      0.194557      
h(out_prncp_inv-1261.18)*loan_amnt                No      1.23867e-05   
h(1261.18-out_prncp_inv)*loan_amnt                No      -0.000153434  
term_60*loan_amnt                                 No      0.0988976     
term_60*h(1261.18-out_prncp_inv)*loan_amnt        No      -0.00014484   
h(out_prncp_inv-15299.2)*term_60*loan_a

### Interpreting MARS basis functions
Based on the model summary in the previous question, answer the following question. Holding all other predictors constant, what will be the mean increase in `money_made_inv` for a unit increase in `out_prncp_inv`, given that `out_prncp_inv` is in [500, 600], `term` = 36 (months), `loan_amnt` = 1000, and `int_rate` = 0.1?

First, write the basis functions being used to answer the question, and then substitute the values.

Also, which basis function is non-zero for the smallest domain space of `out_prncp_inv`? Also, specify the domain space in which it is non-zero.

*(3 + 2 points)*

# Check this!!!
The basis function of this MARS model is

$y = -15.4464 + 0.194557 \times loan\_amnt - 0.000153434 \times (1261.18-out\_prncp\_inv)\times loan\_amnt + 0.0988976 \times term\_60 \times loan\_amnt -0.00014484 \times term\_60 \times (1261.18-out\_prncp\_inv) \times loan\_amnt + 5.46283 \times 10^{-6} \times (15299.2-out\_prncp\_inv) \times term\_60 \times loan\_amnt$

<br>Substituting in the numbers: 
$y = -15.4464 + 0.194557 \times 1000 - 0.000153434 \times (1261.18-out\_prncp\_inv)\times 1000 + 0.0988976 \times 0 \times 1000 -0.00014484 \times 0 \times (1261.18-out\_prncp\_inv) \times 1000 + 5.46283 \times 10^{-6} \times (15299.2-out\_prncp\_inv) \times 0 \times 1000
\\= -14.3973 + 0.153434 \times  out\_prncp\_inv$

<br> Therefore, for a unit increase in `out_prncp_inv`, the mean increase in `money_made_inv` is 0.153434.

### Feature importance

Find the relative importance of each predictor in the MARS model developed in B.3.3. You may choose any criterion for finding feature importance based on the [MARS documentation](https://contrib.scikit-learn.org/py-earth/content.html#multivariate-adaptive-regression-splines). Print a DataFrame with 2 columns - one column consisting of predictors arranged in descending order of relative importance, and the second column quantifying their relative importance. Exclude predictors rejected by the model developed in B.3.3. 

*Note the forward pass and backward passes of the algorithm perform feature selection without manual intervention.*

*(4 points)*

In [ ]:
mars1.summary_feature_importances()

data = list(zip(X_train.columns, mars1.feature_importances_))
importance_df = pd.DataFrame(data, columns =['Feature', 'gcv'])
importance_df.sort_values('gcv', ascending = False)
importance_df.loc[importance_df.gcv !=0]

,Feature,gcv
11,int_rate,0.014617
13,loan_amnt,0.038445
19,out_prncp_inv,0.940297
111,term_60,0.006641


### Prediction
Using the model developed in B.3.3, compute the RMSE on test data.

*(2 points)*

In [ ]:
y_pred = mars1.predict(X_test)
rmse(y_test, y_pred)

648.1808260542323

### Non-trivial train data {-}
Let us call the part of the dataset where `out_prncp_inv = 0` as a trivial subset of data. For this subset, we can directly predict the response without developing a model *(recall the EDA last quarter)*. For all the questions below, fit / tune the  model only on the non-trivial part of the train data. However, when making predictions, and computing RMSE, consider the entire test data. Combine the predictions of the model on the non-trivial subset of test data with the predictions on the trivial subset of test data to make predictions on the entire test data.

In [ ]:
# split the trivial and non-trivial datasets
trivial_X_train = X_train.loc[X_train.out_prncp_inv == 0]
trivial_y_train = y_train.loc[X_train.out_prncp_inv == 0]
nontrivial_X_train = X_train.loc[X_train.out_prncp_inv != 0]
nontrivial_y_train = y_train.loc[X_train.out_prncp_inv != 0]
trivial_X_test = X_test.loc[X_test.out_prncp_inv == 0]
nontrivial_X_test = X_test.loc[X_test.out_prncp_inv != 0]

### Prediction with non-trivial train data
Find the optimal degree of the MARS model based on the non-trivial train data, fit the model, and re-compute the RMSE on test data. 

*Note: You should get a lesser RMSE as compared to what you got in B.3.6.*

*(4 points)*

In [ ]:
parameters={'max_degree':[1,2,3,4]}
mars = Earth()
gs = GridSearchCV(mars, parameters,cv=5,scoring = 'neg_root_mean_squared_error')
nontrivialresult=gs.fit(nontrivial_X_train,nontrivial_y_train)

In [ ]:
df=pd.DataFrame(gs.cv_results_['params'])
df['mean_test_score']=gs.cv_results_['mean_test_score']
df['std_test_score']=gs.cv_results_['std_test_score']
df['rank_test_score']=gs.cv_results_['rank_test_score']
df.sort_values('rank_test_score')

,max_degree,mean_test_score,std_test_score,rank_test_score
3,4,-829.013635,52.434237,1
2,3,-874.929358,100.841665,2
1,2,-896.526608,55.011882,3
0,1,-1263.576021,78.363869,4


In [ ]:
mars2 = Earth(max_degree = 4)
mars2.fit(nontrivial_X_train,nontrivial_y_train)

Earth(max_degree=4)

In [ ]:
non_trivial_y_pred = pd.Series(data=mars2.predict(nontrivial_X_test), index=nontrivial_X_test.index)

In [ ]:
trivial_y_pred = pd.Series(data=0, index=trivial_X_test.index)
trivial_y_pred

2899    0
104     0
3735    0
3668    0
1179    0
       ..
3794    0
757     0
4040    0
1356    0
301     0
Length: 659, dtype: int64

In [ ]:
y_pred_2 = pd.Series(data = None, index = y_test.index)
y_pred_2.loc[trivial_y_pred.index] = trivial_y_pred
y_pred_2.loc[non_trivial_y_pred.index] = non_trivial_y_pred
y_pred_2

<ipython-input-19-e2d6357d2ba7>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_pred_2 = pd.Series(data = None, index = y_test.index)


2899        0.000000
104         0.000000
3735        0.000000
5713   -15262.927639
3668        0.000000
            ...     
757         0.000000
1796      402.747103
4040        0.000000
1356        0.000000
301         0.000000
Length: 1156, dtype: float64

In [ ]:
rmse(y_test,y_pred_2)

484.27809336851317

RMSE on test data is reduced from 648.18 to 484.28.

### Reducing model variance

The MARS model is highly flexible, which makes it a low bias-high variance model. However, high prediction variance increases the expected mean squared error on test data *(see **equation 2.7 on page 34** of the book)*. How can you reduce the prediction variance of the model without increasing the bias? Check slide 12 of the [bias-variance presentation](https://nuwildcat-my.sharepoint.com/personal/akl0407_ads_northwestern_edu/_layouts/15/onedrive.aspx?ga=1&id=%2Fpersonal%2Fakl0407%5Fads%5Fnorthwestern%5Fedu%2FDocuments%2FSTAT303%2D3%20presentations%2FWeek1%5FBiasVariance%2Epdf&parent=%2Fpersonal%2Fakl0407%5Fads%5Fnorthwestern%5Fedu%2FDocuments%2FSTAT303%2D3%20presentations). The MARS model, in general, corresponds to case B. You can see that by averaging the predictions of multiple models, you will reduce prediction variance without increasing the bias.

Take 10 samples of train data of the same size as the train data, with replacement. For each sample, fit a MARS model with the optimal degree identified earlier. Use the $i^{th}$ model, say $\hat{f}_i$ to make prediction $\hat{f_i}(\mathbf{x}_{test})$  on each test data point $\mathbf{x}_{test}$ *(Note that predictions will be made using the model on the non-trivial test data, and without the model on the trivial test data)*. Compute the average prediction on each test data point based on the 10 models as follows: 

$$\hat{f}(\mathbf{x}_{test}) = \frac{1}{10}\Sigma_{1=1}^{10} \hat{f_i}(\mathbf{x}_{test})$$

Consider $\hat{f}(\mathbf{x}_{test})$ as the prediction at the test data point $\mathbf{x}_{test}$. Compute the RMSE based on this model. which is the average prediction of 10 models. You should get a lesser RMSE as compared to the previous question (B.3.7).

*Note: For ease in grading, use the Pandas DataFrame method [`sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) to take samples with replacement, and put `random_state` for the ith sample as i, where i goes from 0 to 9.*

*(6 points)*

In [ ]:
y_pred_all = pd.DataFrame(data = None, index = non_trivial_y_pred.index, columns = range(0,10))

In [ ]:
#nontrivial_X_train,nontrivial_y_train
for s in range(0,10):
  X_train_sample = nontrivial_X_train.sample(frac=1, random_state=s, replace=True, ignore_index=False)
  y_train_sample = nontrivial_y_train.sample(frac=1, random_state=s, replace=True, ignore_index=False)
  mars_sample = Earth(max_degree = 4)
  mars_sample.fit(X_train_sample,y_train_sample)
  y_pred = mars_sample.predict(nontrivial_X_test)
  y_pred_all.loc[:,s] = y_pred

In [ ]:
y_pred_ave = y_pred_all.mean(axis = 1)
y_pred_3 = pd.Series(data = None, index = y_test.index)
y_pred_3.loc[trivial_y_pred.index] = trivial_y_pred
y_pred_3.loc[y_pred_ave.index] = y_pred_ave
#y_pred_3
rmse(y_test, y_pred_3)

<ipython-input-23-5ee0735f000a>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_pred_3 = pd.Series(data = None, index = y_test.index)


354.1392586075983

So the rmse with the reduced-variance model decreases even further to 354.14.

### Generalized additive model (GAM)

Develop a Generalized linear model $\hat{f}_{GLM}(.)$ to predict `money_made_inv` as follows:

$$\hat{f}_{GLM}(\mathbf{x}) = \hat{\beta}_0 + \Sigma_{i=1}^{4} \hat{\beta}_i{f}_i(\mathbf{x}),$$

where ${f}_i(\mathbf{x})$ is a MARS model of degree $i$.

Print the estimated beta coefficients ($\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2, \hat{\beta}_3, \hat{\beta}_4$) of the developed model.

*Note: The model is developed on the non-trivial train data*

*(8 points)*

In [ ]:
# mars model degree = 1,2,3,4 on train data
# get a vector 
def MARSSS(i, X = nontrivial_X_train, y = nontrivial_y_train):
  marsss = Earth(max_degree = i)
  marsss.fit(X,y)
  pred_y = marsss.predict(nontrivial_X_train)
  return pred_y

In [ ]:
gam_train_nontrivial = pd.DataFrame(data = None, index = nontrivial_y_train.index, columns = [1,2,3,4])

In [ ]:
for i in range(1,5):
  gam_train_nontrivial[i] = MARSSS(i)

In [ ]:
# the new training dataset for gam
gam_train_nontrivial.head()

,1,2,3,4
5386,-26209.705659,-27239.889354,-26676.152738,-26676.152738
2860,-9763.741034,-9896.891759,-9806.925538,-9806.925538
547,-3652.218467,-2969.644360,-3240.869947,-3240.869947
3509,-12996.286662,-13406.391605,-13273.717875,-13273.717875
5480,-7990.269186,-8059.371447,-7897.658539,-7897.658539


In [ ]:
# fit the linear model using 4 MARS training matrix
from sklearn.linear_model import LinearRegression
gam_model = LinearRegression()
gam_model.fit(gam_train_nontrivial, nontrivial_y_train)

LinearRegression()

In [ ]:
coeffs = pd.DataFrame(data = [gam_model.coef_], columns = ['beta_1','beta_2','beta_3','beta_4'])
coeffs['beta_0 (intercept)'] = gam_model.intercept_
coeffs

,beta_1,beta_2,beta_3,beta_4,beta_0 (intercept)
0,-0.220107,0.707409,1.697152e+12,-1.697152e+12,-26.295732


### Prediction with GAM

Use the GAM developed in the previous question to compute RMSE on test data.

*Note: Predictions will be made using the model on the non-trivial test data, and without the model on the trivial test data*

*(5 points)*

In [ ]:
# reform the test dataset
def MARSSS_for_test(i, X_test = nontrivial_X_test, X_train = nontrivial_X_train, y_train = nontrivial_y_train):
  marsss = Earth(max_degree = i)
  marsss.fit(X_train,y_train)
  pred_y = marsss.predict(X_test)
  return pred_y

In [ ]:
gam_test_nontrivial = pd.DataFrame(data = None, index = nontrivial_X_test.index, columns = [1,2,3,4])
for i in range(1,5):
  gam_test_nontrivial[i] = MARSSS_for_test(i)

In [ ]:
non_trivial_y_pred = pd.Series(data = gam_model.predict(gam_test_nontrivial), index = nontrivial_X_test.index)
y_pred_4 = pd.Series(data = None, index = y_test.index)
y_pred_4.loc[trivial_y_pred.index] = trivial_y_pred
y_pred_4.loc[non_trivial_y_pred.index] = non_trivial_y_pred
# rmse of the GAM model
rmse(y_test, y_pred_4)

<ipython-input-32-dbd5597d7c38>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_pred_4 = pd.Series(data = None, index = y_test.index)


461.6985165226154

### Reducing GAM prediction variance

As we reduced the variance of the MARS model in B.3.8, follow the same approach to reduce the variance of the GAM developed in B.3.9, and compute the RMSE on test data.

*Note: You should get a lesser RMSE as compared to what you got in B.3.10.*

*(8 points)*

In [ ]:
y_pred_all_gam = pd.DataFrame(data = None, index = non_trivial_y_pred.index, columns = range(0,10))

In [ ]:
#nontrivial_X_train,nontrivial_y_train

for s in range(0,10):
  X_train_sample = nontrivial_X_train.sample(frac=1, random_state=s, replace=True, ignore_index=False)
  y_train_sample = nontrivial_y_train.sample(frac=1, random_state=s, replace=True, ignore_index=False)
  gam_train_nontrivial_sample = pd.DataFrame(data = None, index = X_train_sample.index, columns = [1,2,3,4])
  gam_test_nontrivial_sample = pd.DataFrame(data = None, index = nontrivial_X_test.index, columns = [1,2,3,4])
  for i in range(1,5):
    # training dataset making 4 mars models
    gam_train_nontrivial_sample[i] = MARSSS(i, X = X_train_sample, y = y_train_sample)
    # converting the test dataset
    gam_test_nontrivial_sample[i] = MARSSS_for_test(i, X_test = nontrivial_X_test, X_train = X_train_sample, y_train = y_train_sample)
  gam_model = LinearRegression()
  gam_model.fit(gam_train_nontrivial_sample, nontrivial_y_train)
  y_pred = gam_model.predict(gam_test_nontrivial_sample)
  # put the predicted y's to the dataframe
  y_pred_all_gam.loc[:,s] = y_pred


In [ ]:
y_pred_ave_gam = y_pred_all_gam.mean(axis =1)
y_pred_5 = pd.Series(data = None, index = y_test.index)
y_pred_5.loc[trivial_y_pred.index] = trivial_y_pred
y_pred_5.loc[non_trivial_y_pred.index] = y_pred_ave_gam
# rmse of the GAM model
rmse(y_test, y_pred_5)

<ipython-input-35-2ed016ff7ba8>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_pred_5 = pd.Series(data = None, index = y_test.index)


358.705432641824

The rmse reduces from 461.91 in GAM without reducing variance to 358.71.

### Natural cubic splines

Even though MARS is efficient and highly flexible, natural cubic splines work very well too, if tuned properly.

Consider the predictors identified in the model summary of the MARS model printed in B.3.3. For each predictor, create natural cubic splines basis functions with $d$ degrees of freedom. Include all-order interactions *(i.e., 2-factor, 3-factor, 4-factor interactions, and so on)* of all the basis functions. Use the `sklearn` function `cross_val_score()` to find and report the optimal degrees of freedom for the natural cubic spline of each predictor. 

Consider degrees of freedom from 3 to 6 for the natural cubic spline transformation of each predictor.

*(8 points)*

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val = pd.DataFrame(columns = ['d_out', 'd_loan','d_int','d_term','rmse'])
counter = 0
for d_out in range(3,7):
  for d_loan in range(3,7):
    for d_int in range(3,7):
      for d_term in range(3,7):
        X_train_poly = dmatrix('cr(out_prncp_inv, df = ' + str(d_out) + ')*cr(loan_amnt, df =' +str(d_loan) +')* \
        cr(int_rate, df =' + str(d_int) + ')*cr(term_60, df =' + str(d_term) +')', 
        data = nontrivial_X_train, return_type = 'dataframe')

        cross_val.loc[counter, 'rmse'] = np.sqrt(np.mean(-cross_val_score(LinearRegression(), 
                                                                          X_train_poly, nontrivial_y_train, scoring = 'neg_mean_squared_error', cv = 5, n_jobs = -1)))
        cross_val.loc[counter, 'd_out'] = d_out
        cross_val.loc[counter, 'd_loan'] = d_loan
        cross_val.loc[counter, 'd_int'] = d_int
        cross_val.loc[counter, 'd_term'] = d_term
        counter = counter + 1

In [ ]:
cross_val.loc[cross_val.rmse == cross_val.rmse.min(),:]

,d_out,d_loan,d_int,d_term,rmse
6,3,3,4,5,466.002877


### Fitting the natural cubic splines model

With the optimal degrees of freedom identified in the previous question, fit a model to predict `money_made_inv`, where the basis functions correspond to the natural cubic splines of each predictor, and all-factor interactions of the basis functions. Compute the RMSE on test data.

*Note: Predictions will be made using the model on the non-trivial test data, and without the model on the trivial test data*

*(4 points)*

In [ ]:
# to use statsmodels, x and y should be in the same frame
nontrivial_train = pd.concat([nontrivial_X_train, nontrivial_y_train], axis = 1)
nontrivial_train.head()

,id,acc_now_delinq,acc_open_past_24mths,annual_inc,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,earliest_cr_line,...,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E4,sub_grade_Other_grade,term_60,verification_status_Source Verified,verification_status_Verified,money_made_inv
5386,9205,0,4,135000.0,3957,89.1,0,0,10.62,1001894400000000000,...,0,0,0,0,0,0,1,1,0,-27880.03
2860,6679,0,2,80000.0,6778,95.0,0,0,6.71,1427846400000000000,...,0,0,0,0,0,0,0,1,0,-10279.94
547,4366,0,6,78500.0,30785,50.6,0,0,7.05,1078099200000000000,...,0,0,0,0,0,0,0,0,0,-3060.12
3509,7328,0,1,84000.0,4939,73.7,0,0,23.60,1283299200000000000,...,0,0,0,0,0,0,0,1,0,-12621.26
5480,9299,0,3,82000.0,10657,49.9,0,0,28.82,1009843200000000000,...,0,0,0,0,0,0,0,1,0,-8083.84


In [ ]:
reg_spline_model = smf.ols('money_made_inv~cr(out_prncp_inv, df = 3)*cr(loan_amnt, df =3)* \
        cr(int_rate, df =4)*cr(term_60, df =5)', data = nontrivial_train).fit()

In [ ]:
non_trivial_y_pred = pd.Series(data = reg_spline_model.predict(nontrivial_X_test), index = nontrivial_X_test.index)
y_pred_6 = pd.Series(data = None, index = y_test.index)
y_pred_6.loc[trivial_y_pred.index] = trivial_y_pred
y_pred_6.loc[non_trivial_y_pred.index] = non_trivial_y_pred
# rmse of the spline model
rmse(y_test, y_pred_6)

<ipython-input-45-4508702b49cd>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_pred_6 = pd.Series(data = None, index = y_test.index)


183.87785088634226

## GAM for classification
The data for this question is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls, where bank clients were called to subscribe for a term deposit. 

There is one train data - *train.csv*, which you will use to develop a model. There are two test datasets - *test1.csv* and *test2.csv*, which you will use to test your model. Each dataset has the following attributes about the clients called in the marketing campaign:

1. `age`: Age of the client

2. `education`: Education level of the client 

3. `day`: Day of the month the call is made

4. `month`: Month of the call 

5. `y`: did the client subscribe to a term deposit? 

6. `duration`: Call duration, in seconds. This attribute highly affects the output target (e.g., if `duration`=0 then `y`='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call `y` is obviously known. Thus, this input should only be included for inference purposes and should be discarded if the intention is to have a realistic predictive model.

(Raw data source: [Source](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Do not use the raw data source for this assingment. It is just for reference.)

Develop a **generalized additive model (GAM)** to predict the probability of a client subscribing to a term deposit based on *age, education, day* and *month*. The model must have: \
(a)  **Minimum overall classification accuracy of 75%** among the classifcation accuracies on *train.csv*, *test1.csv* and *test2.csv*. \
(b) **Minimum recall of 55%** among the recall on *train.csv*, *test1.csv* and *test2.csv*. 

Print the accuracy and recall for all the three datasets - *train.csv*, *test1.csv* and *test2.csv*.

Note that: 

i. You cannot use `duration` as a predictor. The predictor is not useful for prediction because its value is determined after the marketing call ends. However, after the call ends, we already know whether the client responded positively or negatively. 

ii. One way to develop the model satisfying constrains (a) and (b) is to use **spline tranformations for *age* and *day*, and interacting *month* with all the predictors (including the spline transformations)**

iii. You may assume that the distribution of the predictors is the same in all the three datasets. Thus, you may create B-spline basis functions independently for the train and test datasets.

iv. Use cross-validation on train data to optimize the model hyperparameters, and the decision threshold probability. Then, use the optimal hyperparameters to fit the model on train data. Then, evaluate its accuracy and recall on all the three datasets. Note that the test datasets must only be used to evaluate performance metrics, and not optimize any hyperparameters or decision threshold probability.

*(20 points: 10 points for cross validation, 5 points for obtaining and showing the optimal values of the hyperparameters and decision threshold probability, 2 points for fitting the model with the optimal hyperparameters, and 3 points for printing the accuracy & recall on each of the three datasets)*

In [2]:
train = pd.read_csv('train.csv')
test1 = pd.read_csv('test1.csv')
test2 = pd.read_csv('test2.csv')

In [3]:
# drop duration
train.drop(columns = ['duration'], inplace = True)
test1.drop(columns = ['duration','default'], inplace = True)
test2.drop(columns = ['duration'], inplace = True)

In [4]:
train_dum = pd.get_dummies(train,drop_first = True)
test1_dum = pd.get_dummies(test1,drop_first = True)
test2_dum = pd.get_dummies(test2,drop_first = True)

In [5]:
X_train_num = train[['age','day']]
X_test1_num = test1[['age','day']]
X_test2_num = test2[['age','day']]

In [6]:
X_train = train_dum.drop(columns = 'y_yes')
X_test1 = test1_dum.drop(columns = 'y_yes')
X_test2 = test2_dum.drop(columns = 'y_yes')
y_train = train_dum['y_yes']
y_test1 = test1_dum['y_yes']
y_test2 = test2_dum['y_yes']

In [7]:
# scale the X_num
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.set_output(transform = 'pandas')
scaler.fit(X_train_num)
X_train_std = scaler.transform(X_train_num)
X_test1_std = scaler.transform(X_test1_num)
X_test2_std = scaler.transform(X_test2_num)
#X_train_std = pd.DataFrame(

In [8]:
#X_train[['age','day']] = X_train_std
#X_test1[['age','day']] = X_test1_std
#X_test2[['age','day']] = X_test2_std

In [9]:
# make the string for the formula
string = ''
for stri in X_train.columns[2:5]:
  string = string +'*'+ stri 
for stri in X_train.columns[5:]:
  string = string +'+'+ stri 

In [10]:
X_train

,age,day,education_secondary,education_tertiary,education_unknown,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,42,15,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,37,20,1,0,0,0,0,0,0,0,1,0,0,0,0,0
2,32,17,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,53,28,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,32,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,32,30,1,0,0,0,0,0,0,0,0,0,1,0,0,0
34996,32,11,0,1,0,0,0,0,0,0,0,1,0,0,0,0
34997,56,19,0,1,0,0,0,0,0,0,0,0,0,1,0,0
34998,30,17,1,0,0,0,0,0,0,0,1,0,0,0,0,0


In [52]:
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
predict_result = pd.DataFrame(columns = ['d_age', 'd_day', 'pred_value'])
counter = 0
for d_age in range(5,10):
  for d_day in range(5,10):
      X_train_poly = dmatrix('cr(age, df = ' + str(d_age) + ')*cr(day, df =' +str(d_day)  +')'+ string,
        data = X_train, return_type = 'dataframe')
      predict_result.loc[counter, 'pred_value'] = cross_val_predict(LogisticRegression(), X_train_poly, y_train,  cv = 5, n_jobs = -1, method = 'predict_proba')
      predict_result.loc[counter, 'd_age'] = d_age
      predict_result.loc[counter, 'd_day'] = d_day
      counter = counter + 1

In [12]:
len(predict_result.iloc[0,2])

35000

In [53]:
predict_result

,d_age,d_day,pred_value
0,5,5,"[[0.9506317054103779, 0.04936829458962214], [0..."
1,5,6,"[[0.9487506228619054, 0.0512493771380946], [0...."
2,5,7,"[[0.9500042328575833, 0.04999576714241673], [0..."
3,5,8,"[[0.9398217026810296, 0.060178297318970383], [..."
4,5,9,"[[0.9449337004041383, 0.05506629959586175], [0..."
5,6,5,"[[0.9486443422175919, 0.051355657782408096], [..."
6,6,6,"[[0.9468974335637486, 0.05310256643625146], [0..."
7,6,7,"[[0.9488669191954429, 0.05113308080455713], [0..."
8,6,8,"[[0.9387249703406957, 0.061275029659304286], [..."
9,6,9,"[[0.9449080599795998, 0.0550919400204002], [0...."


In [54]:
pd.DataFrame(data = predict_result.iloc[1,2])[1]

0        0.051249
1        0.057697
2        0.186161
3        0.064008
4        0.159862
           ...   
34995    0.053425
34996    0.654626
34997    0.122468
34998    0.110723
34999    0.118021
Name: 1, Length: 35000, dtype: float64

In [55]:
predicted_class_df = pd.Series(data = None, index = range(0,35000))
predicted_class_df = (pd.DataFrame(data = predict_result.iloc[1,2])[1] > 0.1).astype(int)


<ipython-input-55-d583d0e85b96>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  predicted_class_df = pd.Series(data = None, index = range(0,35000))


In [56]:
predicted_class_df

0        0
1        0
2        1
3        0
4        1
        ..
34995    0
34996    1
34997    1
34998    1
34999    1
Name: 1, Length: 35000, dtype: int64

In [57]:
from sklearn.metrics import accuracy_score, recall_score

In [63]:
hyperparam_vals = np.arange(0,1.01,0.01)
acc_recall_table = pd.DataFrame(data = None, columns = ['counter','threshold','accuracy','recall'])
for counter in range(0,25):
  locals()['accuracy_iter_c{0}'.format(counter)] = []
  locals()['recall_iter_c{0}'.format(counter)] = []
  for threshold_prob in hyperparam_vals:
    # original predictions
    predicted_probability_df = pd.DataFrame(data = predict_result.iloc[counter,2])
    # convert to 0 and 1
    predicted_class_df = pd.Series(data = None, index = range(0,35000), dtype = 'float64')
    predicted_class_df = (predicted_probability_df[1] > threshold_prob).astype(int)
    #Computing the accuracy
    accuracy = accuracy_score(y_train, predicted_class_df)*100
    #Computing the recall
    recall = recall_score(y_train, predicted_class_df, zero_division = 0)*100
    locals()['accuracy_iter_c{0}'.format(counter)].append(accuracy)
    locals()['recall_iter_c{0}'.format(counter)].append(recall)


In [71]:
# select out model that will give the satisfying accuracy and recall
for c in range(0,25):
  print('for the ' + str(c)+ 'th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:')
  for i in range(0,51):
    if ((locals()['accuracy_iter_c{0}'.format(c)])[i] >= 75) & ((locals()['recall_iter_c{0}'.format(c)])[i] >= 53):
        print(hyperparam_vals[i])


for the 0th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 1th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 2th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 3th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 4th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 5th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 6th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 7th iteration, the following threshold (by order) will give the accuracy and recall that satisfy the request:
for the 8th iteration, the following threshold (by order

In [72]:
# so now i have multiple combinations to choose. I will choose, for example, the 8th iteration
# with the 37th prediction threshold
print(predict_result.loc[13,:])


d_age                                                         7
d_day                                                         8
pred_value    [[0.9338178318568735, 0.06618216814312654], [0...
Name: 13, dtype: object
prediction threshold: 0.37


In [73]:
# build the model
logis_model = smf.logit(formula = 'y_yes~cr(age, df =7)*cr(day, df =8)'+ string, data = train_dum).fit(iter = 10000, method='lbfgs')

/usr/local/lib/python3.9/dist-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: iter. After release 0.14, this will raise.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [87]:
#y_pred = logis_model.predict()
predicted1 = logis_model.predict(test1_dum)
predicted2 = logis_model.predict(test2_dum)
predicted_choice1 = (predicted1 > 0.13).astype(int)
predicted_choice2 = (predicted2 > 0.13).astype(int)

In [88]:
recall_score(y_test1, predicted_choice1)

0.5202702702702703

In [89]:
recall_score(y_test2, predicted_choice2)

0.5531197301854974

In [90]:
accuracy_score(y_test2, predicted_choice2)

0.7569947172764625

In [91]:
accuracy_score(y_test1, predicted_choice1)

0.7603921568627451

In [ ]:
X_train_poly = dmatrix('cr(age, df = ' + str(4) + ')*cr(day, df =' +str(3)  +')'+ string,
        data = X_train, return_type = 'dataframe')

In [ ]:
X_train_poly

,Intercept,"cr(age, df=4)[0]","cr(age, df=4)[1]","cr(age, df=4)[2]","cr(age, df=4)[3]","cr(day, df=3)[0]","cr(day, df=3)[1]","cr(day, df=3)[2]","cr(age, df=4)[0]:cr(day, df=3)[0]","cr(age, df=4)[1]:cr(day, df=3)[0]",...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,1.0,0.026772,1.005427,-0.038298,0.006099,0.036593,0.993481,-0.030074,0.000980,0.036791,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.162613,0.946198,-0.129422,0.020610,-0.084741,0.902815,0.181926,-0.013780,-0.080182,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.345138,0.782154,-0.151404,0.024111,-0.030074,0.993481,0.036593,-0.010380,-0.023523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,-0.080425,0.719060,0.419289,-0.057924,-0.048000,0.296000,0.752000,0.003860,-0.034515,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.345138,0.782154,-0.151404,0.024111,0.916741,0.099852,-0.016593,0.316402,0.717033,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,1.0,0.345138,0.782154,-0.151404,0.024111,-0.016593,0.099852,0.916741,-0.005727,-0.012978,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34996,1.0,0.345138,0.782154,-0.151404,0.024111,0.240741,0.851852,-0.092593,0.083089,0.188296,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34997,1.0,-0.075387,0.577320,0.568731,-0.070664,-0.072000,0.944000,0.128000,0.005428,-0.041567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34998,1.0,0.428469,0.693367,-0.144913,0.023077,-0.030074,0.993481,0.036593,-0.012886,-0.020852,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_poly

,Intercept,"cr(age, df=4)[0]","cr(age, df=4)[1]","cr(age, df=4)[2]","cr(age, df=4)[3]","cr(day, df=5)[0]","cr(day, df=5)[1]","cr(day, df=5)[2]","cr(day, df=5)[3]","cr(day, df=5)[4]",...,"cr(age, df=4)[2]:cr(day, df=5)[2]","cr(age, df=4)[3]:cr(day, df=5)[2]","cr(age, df=4)[0]:cr(day, df=5)[3]","cr(age, df=4)[1]:cr(day, df=5)[3]","cr(age, df=4)[2]:cr(day, df=5)[3]","cr(age, df=4)[3]:cr(day, df=5)[3]","cr(age, df=4)[0]:cr(day, df=5)[4]","cr(age, df=4)[1]:cr(day, df=5)[4]","cr(age, df=4)[2]:cr(day, df=5)[4]","cr(age, df=4)[3]:cr(day, df=5)[4]"
0,1.0,0.026772,1.005427,-0.038298,0.006099,-0.019672,0.120402,0.964614,-0.078413,0.013069,...,-0.036943,0.005883,-0.002099,-0.078838,0.003003,-0.000478,0.000350,0.013140,-0.000501,0.000080
1,1.0,0.162613,0.946198,-0.129422,0.020610,0.019259,-0.115556,0.563852,0.608593,-0.076148,...,-0.072975,0.011621,0.098965,0.575849,-0.078765,0.012543,-0.012383,-0.072051,0.009855,-0.001569
2,1.0,0.345138,0.782154,-0.151404,0.024111,0.013069,-0.078413,0.964614,0.120402,-0.019672,...,-0.146046,0.023258,0.041555,0.094173,-0.018229,0.002903,-0.006790,-0.015387,0.002978,-0.000474
3,1.0,-0.080425,0.719060,0.419289,-0.057924,-0.006000,0.036000,-0.144000,0.604000,0.510000,...,-0.060378,0.008341,-0.048577,0.434312,0.253250,-0.034986,-0.041017,0.366721,0.213837,-0.029541
4,1.0,0.345138,0.782154,-0.151404,0.024111,0.831587,0.212847,-0.056127,0.014032,-0.002339,...,0.008498,-0.001353,0.004843,0.010975,-0.002124,0.000338,-0.000807,-0.001829,0.000354,-0.000056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,1.0,0.345138,0.782154,-0.151404,0.024111,-0.002339,0.014032,-0.056127,0.212847,0.831587,...,0.008498,-0.001353,0.073462,0.166479,-0.032226,0.005132,0.287013,0.650430,-0.125906,0.020050
34996,1.0,0.345138,0.782154,-0.151404,0.024111,-0.078042,0.764550,0.386243,-0.087302,0.014550,...,-0.058479,0.009313,-0.030131,-0.068283,0.013218,-0.002105,0.005022,0.011381,-0.002203,0.000351
34997,1.0,-0.075387,0.577320,0.568731,-0.070664,0.021429,-0.128571,0.730286,0.439429,-0.062571,...,0.415336,-0.051605,-0.033127,0.253691,0.249917,-0.031052,0.004717,-0.036124,-0.035586,0.004422
34998,1.0,0.428469,0.693367,-0.144913,0.023077,0.013069,-0.078413,0.964614,0.120402,-0.019672,...,-0.139785,0.022261,0.051589,0.083483,-0.017448,0.002779,-0.008429,-0.013640,0.002851,-0.000454


In [ ]:
for i in range(1,6):
    locals()['train_f{0}'.format(i)] = train.loc[locals()['f{0}'.format(i)][0],:]
    locals()['val_f{0}'.format(i)] = train.loc[locals()['f{0}'.format(i)][1],:]
train_f1
val_f5

In [ ]:
from sklearn.metrics import accuracy_score